# Torus one-point functions of the $O(n)$ model

In [1]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools, BarnesDoubleGamma
println("Number of threads: $(Threads.nthreads())")

Number of threads: 64


In [3]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2") / (big"0.8" + big"0.2" * im))
diag_field = Field(c, r=0, s=big"0.4" + big"0.1" * im)
ndiag_indices = [(r, s) for r in 1//2:1//2:40 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields=nothing; c=CC(β=sqrt(big"2") / (big"0.8" + big"0.2" * im)),
    P=big"0.4" + big"0.1" * im, parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 0.75 * precision)
    if fields === nothing
        diag_field = Field(c, P=P)
        ndiag_indices = [(r, s) for r in 1//2:1//2:40 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], diag_field)
    end
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity == 1 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    parity == -1 && (fields = filter(V -> spin(V) != 0 && 0 < V.s && V.s < 1, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    rmax=3, show=true,
    Ps=nothing, diagchan=nothing,
    fix=nothing,
    rels=:stu
)
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels) # rels = :stu: structure constants equal in all channels.
        if show
            Base.show(stdout, res.str_cst.s, rmax=rmax) # print only s-channel, since others are equal.
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end;

# Count dimensions of spaces of solutions

| r                       | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 |
|-------------------------|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|
|number of solutions      | 1 | 1 | 2 | 3 | 4 | 5 | 8 | 9 |12 |15 | 18 | 21 | 26 | 29 | 34 |
|number of even solutions | 1 | 1 | 2 | 3 | 4 | 5 | 7 | 8 |10 |12 | 14 | 16 | 19 | 20 |    |
|number of odd  solutions | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 |2  |3  | 4  | 5  | 7  | 9  |    |

In [19]:
fixed_constants_order = Dict(
    0 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1]]...,],
    1 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1] if V.s >= 0]...,],
   -1 => [[:s, V, 0] for V in fields[1:end-1] if spin(V) != 0 && 0 < V.s && V.s < 1],
)
fixed_constants_order[-1][1][3] = 1

nb_comb_maps = Dict(
    0 => [1, 2, 3, 4, 5, 8, 9, 12, 15, 18, 21, 26, 29, 34, 39], # number of solutions
    1 => [1, 2, 3, 4, 5, 7, 8, 10, 12, 14, 16, 19, 20], # number of parity-even solutions
   -1 => [0, 0, 0, 0, 0, 1, 1, 2, 3, 4, 5, 7, 9,], # number of parity-odd solutions
)

precisions = [40, 40, 40, 40, 40, 40, 60, 65, 75, 85, 90, 95, 100, 130]

function check_dimension(r, s; parity=0, expected_dimension=nothing, not_fixed = [], precision=nothing, blocks=nothing)
    printstyled("=====\nr = $r\n=====\n", bold=true)
    (expected_dimension === nothing) && (expected_dimension = nb_comb_maps[parity][r])
    (precision === nothing) && (precision = precisions[r])
    (blocks === nothing) && (blocks = precompute_blocks((r, s), fields, parity=parity, precision=precisions[r]))
    fixed = [f for f in fixed_constants_order[parity][1:expected_dimension + length(not_fixed)] if !(f in not_fixed)]
    println("With $(length(fixed)) constants fixed:")
    solve(blocks, Sig(0, 0, 0), fix=fixed, rmax=(r + 3) // 3)
    if length(fixed) > 1
        println("With $(length(fixed)-1) constant fixed:")
        solve(blocks, Sig(0, 0, 0), fix=fixed[1:end-1], rmax=(r + 3) // 3)
    end
end;

## Parity unfixed

### $r \in \{1, \dots, 8\} \cup \{ 10\}$

In [59]:
@time for r in [1:8..., 10]
    check_dimension(r, 0)
end

=====
r = 1
=====
With 1 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 6.2154-4.35819im   │ 1.5e-44   │
│ (1, 0)             │ 5.84666-10.1272im  │ 8.2e-43   │
│ (1, 1)             │ -4.14065-1.37768im │ 5.2e-43   │
└────────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 0.0+0.0im          │ 0         │
│ (1, 0)             │ -934.438+210.945im │ 9.4e-45   │
│ (1, 1)             │ -597.189+296.177im │ 1.9e-44   │
│ (3//2, 0)          │ 1538.45+4136.88im  │ 2.7e-41   │
│ (3//2, -2//3)     

### $r = 9$

In [27]:
check_dimension(9, 0, not_fixed=fixed_constants_order[0][14:15]);

=====
r = 9
=====
With 15 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 1.19433e+25-1.32957e+25im  │ 1.9e-41  

Note that with the first 14 constants fixed, the two remaining solutions both have the same $D_{(\frac72, s)}$ and $D_{(\frac52, s)}$, but not the same $D_{(3, s)}$.

### $r=11$

In [34]:
r = 11
blocks = precompute_blocks((r, 0), fields, parity=0, precision=precisions[r]);

In [35]:
check_dimension(11, 0, blocks=blocks);

=====
r = 11
=====
With 21 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

If we fix the first 20 constants, $D_{(\frac32, \frac23)} = 0$ in all the remaining solutions. So we need to set something else to zero. The following works (the two red lines are zero):

In [36]:
check_dimension(11, 0, blocks=blocks, not_fixed=fixed_constants_order[0][19:20], expected_dimension=21);

=====
r = 11
=====
With 21 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=12$

In [52]:
check_dimension(12, 0, not_fixed=fixed_constants_order[0][26:29]);

=====
r = 12
=====
With 26 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=13$

In [58]:
check_dimension(13, 0);

=====
r = 13
=====
With 29 constants fixed:
┌────────────────────┬──────────────────────────────┬───────────┐
│ Field              │ Structure constant           │ Rel. err. │
├────────────────────┼──────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                    │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (1, 0)             │ 0.0+0.0im                    │ 0         │
│ (1, 1)             │ 0.0+0.0im                    │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                    │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                    │ 0         │
│ (2, 0)             │ 0.0+0.0im                    │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                    │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                    │ 0         │
│ (2, 1)             │ 0.0+0.0im                    │ 0         │
│ (5//2, 0)          │ 0.0+0.0im

In [74]:
fixed_constants_order[0][26:29]

4-element Vector{Tuple{Symbol, Field{Complex{BigFloat}}, Int64}}:
 (:s, (7//2, 0), 0)
 (:s, (7//2, 2//7), 0)
 (:s, (7//2, 4//7), 0)
 (:s, (7//2, 6//7), 0)

In [55]:
check_dimension(13, 0, not_fixed=fixed_constants_order[0][26:29]);

=====
r = 13
=====
With 29 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0       

### $r=14$

In [57]:
check_dimension(14, 0, not_fixed=fixed_constants_order[0][26:32]);

=====
r = 14
=====
With 34 constants fixed:
┌────────────────────┬──────────────────────────────┬───────────┐
│ Field              │ Structure constant           │ Rel. err. │
├────────────────────┼──────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                    │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (1, 0)             │ 0.0+0.0im                    │ 0         │
│ (1, 1)             │ 0.0+0.0im                    │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                    │ 0         │
│ (3//2, -2//3)      │ 0.0+0.0im                    │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                    │ 0         │
│ (2, 0)             │ 0.0+0.0im                    │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                    │ 0         │
│ (2, -1//2)         │ 0.0+0.0im                    │ 0         │
│ (2, 1)             │ 0.0+0.0im                    │ 0         │
│ (5//2, 0)          │ 0.0+0.0im

## Parity-even solutions

### $r \in \{1, \dots, 8\} \cup \{10\}$

In [ ]:
for r = [1:8..., 10]
    check_dimension(r, 0, parity=1)
end

=====
r = 1
=====
With 1 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 6.2154-4.35819im   │ 8.1e-46   │
│ (1, 0)             │ 5.84666-10.1272im  │ 4.1e-44   │
│ (1, 1)             │ -4.14065-1.37768im │ 6.3e-44   │
└────────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌────────────────────┬────────────────────┬───────────┐
│ Field              │ Structure constant │ Rel. err. │
├────────────────────┼────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)          │ 0.0+0.0im          │ 0         │
│ (1, 0)             │ -934.438+210.945im │ 2.8e-45   │
│ (1, 1)             │ -597.189+296.177im │ 1.6e-44   │
│ (3//2, 0)          │ 1538.45+4136.88im  │ 1e-41     │
│ (3//2, 2//3)      

### $r=9$

In [32]:
check_dimension(9, 0, parity=1);

=====
r = 9
=====
With 12 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0        

Again we observe a relation between the $D_{(\frac52, s)}$ once we fix the $D_{(r\leq 2, s)}$. We get a unique solution if we fix one of the $D_{(3, s)}$:

In [37]:
check_dimension(9, 0, parity=1, expected_dimension = 12, not_fixed=fixed_constants_order[1][10:10]);

=====
r = 9
=====
With 12 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 1.19433e+25-1.32957e+25im  │ 1e-40     │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0        

### $r=11$

In [ ]:
r = 11
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][15:16]);

=====
r = 11
=====
With 16 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

### $r=12$

In [68]:
r = 12
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][19:20]);

=====
r = 12
=====
With 19 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

### $r=13$

In [79]:
r = 13
check_dimension(r, 0, parity=1, not_fixed=fixed_constants_order[1][17:22]);

=====
r = 13
=====
With 20 constants fixed:
┌────────────────────┬────────────────────────────┬───────────┐
│ Field              │ Structure constant         │ Rel. err. │
├────────────────────┼────────────────────────────┼───────────┤
│ P = (0.11+0.057im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 0)             │ 0.0+0.0im                  │ 0         │
│ (1, 1)             │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)       │ 0.0+0.0im                  │ 0         │
│ (2, 0)             │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)          │ 0.0+0.0im                  │ 0         │
│ (2, 1)             │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5)       │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5)       │ 0.0+0.0im                  │ 0       

## Parity-odd solutions

In [34]:
@time for r in 6:12
    check_dimension(r, 0, parity=-1)
end

=====
r = 6
=====
With 1 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ -353400.0+378860.0im       │ 3.9e-29   │
│ (5//2, 2//5) │ -4.93811e+10-2.13787e+11im │ 3e-27     │
│ (5//2, 4//5) │ 8.97642e+10+3.08534e+11im  │ 5.1e-28   │
│ (3, 1//3)    │ -1.50455e+13-4.45181e+12im │ 7.5e-21   │
│ (3, 2//3)    │ -1.22313e+13-1.97343e+12im │ 9.4e-21   │
└──────────────┴────────────────────────────┴───────────┘
=====
r = 7
=====
With 1 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ 0                          │ 3         │
│ (5//2, 2//5) │ 1.14734e+13-9.00323e+12im  │ 

In [32]:
check_dimension(13, 0, parity=-1, not_fixed=fixed_constants_order[-1][9:10]);

=====
r = 13
=====
With 9 constants fixed:
┌──────────────┬────────────────────────────┬───────────┐
│ Field        │ Structure constant         │ Rel. err. │
├──────────────┼────────────────────────────┼───────────┤
│ (3//2, 2//3) │ 1.0+0.0im                  │ 0         │
│ (2, 1//2)    │ 0.0+0.0im                  │ 0         │
│ (5//2, 2//5) │ 0.0+0.0im                  │ 0         │
│ (5//2, 4//5) │ 0.0+0.0im                  │ 0         │
│ (3, 1//3)    │ 0.0+0.0im                  │ 0         │
│ (3, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (7//2, 2//7) │ 0.0+0.0im                  │ 0         │
│ (7//2, 4//7) │ 0.0+0.0im                  │ 0         │
│ (7//2, 6//7) │ 3.06632e+35-3.38827e+34im  │ 3e-12     │
│ (4, 1//4)    │ -3.8057e+47-3.98064e+46im  │ 1.4e-11   │
│ (4, 1//2)    │ 0.0+0.0im                  │ 0         │
│ (4, 3//4)    │ 4.08535e+47+5.92417e+46im  │ 2.3e-11   │
│ (9//2, 2//9) │ 4.1809e+63+3.24559e+63im   │ 2.7e-13   │
│ (9//2, 4//9) │ 6.42907e+63+

# Polynomials

In [35]:
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue

wrs(r, s, β) = 2cospi(r * β^2 - s)

βs = [1 / (big"0.8" + big"0.2" * im) + (i - 1) // 10 for i in 1:10]
cs = [CC(β=β) for β in βs]
DGs = [DoubleGamma(c.β) for c in cs]
inds = [(1, 0) for β in βs]
Ps = [[(big"0.2" + big"0.2" * im + i // 100 * im) * cs[1].β / cs[j].β for i in 1:10] for j = eachindex(cs)] # diag field identical in each channel
blocks = [[precompute_blocks(inds[j], c=c, P=P, parity=1, precision=15) for (i, P) = enumerate(Ps[j])] for (j, c) in enumerate(cs)]
sols = [[solve(blocks[j][i], Sig(0, 0, 0), show=false, rels=:stu) for i = eachindex(Ps[1])] for j = eachindex(cs)];

In [39]:
function compute_times_ref_minus_res(r, s)
    vals = [[sols[j][i].str_cst.s.constants[Field(c, r=r, s=s)] for i = eachindex(Ps[1])] for (j, c) = enumerate(cs)]
    Threads.@threads for j = eachindex(cs)
        c = cs[j]
        DG = DGs[j]
        V = Field(c, r=r, s=s)
        n = -2cospi(c.β^2)
        Threads.@threads for i = eachindex(Ps[j])
            P = Ps[j][i]
            co = sols[j][i].correlation
            diag_field = Field(c, P=P)
            w = 2cospi(2c.β * P)
            DP = compute_reference(co, diag_field, :s, DG)
            DV = compute_reference(co, V, :s, DG)
            vals[j][i] *= DP / DV
            vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
        end
    end
    return [((2cospi(2c.β * Ps[j][i]), -2cospi(c.β^2)), vals[j][i]) for i = eachindex(Ps[1]) for (j, c) = enumerate(cs)]
end

function fit_polynomial(data, varnames, degs)
    if varnames === (:w, :n) || varnames === (:n, :w)
        to_fit = data
    elseif varnames === (:w,)
        to_fit = [((d[1][1],), d[2]) for d in data[1:length(cs):end]]
    elseif varnames === (:n,)
        to_fit = [((d[1][2],), d[2]) for d in data[1:length(cs)]]
    else
        error("Unsupported varnames")
    end
    fit!(Polyfit(varnames, degs), to_fit)
end;

In [ ]:
# only run to reset the data dictionary! a bit expensive to recompute
data = Dict()

In [42]:
data = Dict()
data[1//2, 0] = compute_times_ref_minus_res(1//2, 0);

In [43]:
for deg in 0:2
    display(fit_polynomial(data[1//2, 0], (:w,), (0,)))
end

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[1.0000000000410474172 - 2.7489562962236405541e-11im])

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[1.0000000000410474172 - 2.7489562962236405541e-11im])

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[1.0000000000410474172 - 2.7489562962236405541e-11im])

In [ ]:
data[1, 0] = compute_times_ref_minus_res(1, 0)

In [48]:
for deg in 0:5
    display(fit_polynomial(data[1, 0], (:w,), (deg,)))
end

Polyfit((:w,), (0,), 1, [(0,)], Complex{BigFloat}[3.483519689795254913 - 3.3376492483184574296im])

Polyfit((:w,), (1,), 2, [(0,), (1,)], Complex{BigFloat}[1.5107047935854339225 - 2.1888392549026101305im, 0.052271642278724619337 + 0.50603066785006256718im])

Polyfit((:w,), (2,), 3, [(0,), (1,), (2,)], Complex{BigFloat}[3.6861299242591109328 + 2.5284897860652091363im, 2.4266631725224343669 + 0.46094818803848239307im, 0.10346654085395973384 - 0.23637741222447710538im])

Polyfit((:w,), (3,), 4, [(0,), (1,), (2,), (3,)], Complex{BigFloat}[3.9415767797465264335 - 2.9768521588293981282im, -0.98509757822097194623 - 1.2155333050953222498im, -0.54865973477955138054 + 0.30420700378464773797im, 0.015685583820755427177 + 0.059048598421970815867im])

Polyfit((:w,), (4,), 5, [(0,), (1,), (2,), (3,), (4,)], Complex{BigFloat}[1.0913144569519979028 + 1.9926882494372647244im, 2.1750848206623109408 + 3.0204068181602719487im, 1.1950771479483926901 - 0.092612186644010408357im, 0.068324816449757637079 - 0.19889554999117804984im, -0.011475334338733376565 - 0.0084318098499425765137im])

Polyfit((:w,), (5,), 6, [(0,), (1,), (2,), (3,), (4,), (5,)], Complex{BigFloat}[5.9470317348103356181 - 1.1884937265786482876im, 1.0283259883730847539 - 3.5918236854463543341im, -1.7674324531080893349 - 0.82592065411521292662im, -0.35188485815021227927 + 0.33912098851466502625im, 0.023323435795329984055 + 0.058169100096717645345im, 0.0032540875602599407456 - 0.00016196996744763261071im])

In [50]:
V = Field(c, r=1, s=0)
ρ_residue(V, V)

-49.562719045693664821 + 147.79713032723374266im

In [ ]:
2 * (-2cospi(c.β^2)+2)